In [ ]:
import requests
import json

def get_ollama_models(ip_port):
    """从指定的 Ollama 服务器获取已安装的模型名称"""
    url = f"http://{ip_port}/api/tags"
    
    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()  # 检查 HTTP 是否返回错误
        
        # 解析 JSON
        data = response.json()
        
        # 提取所有模型名称
        models = [model["model"] for model in data.get("models", [])]

        if models:
            print(f"\n在 {ip_port} 发现的 Ollama 模型：")
            for model in models:
                print(f" - {model}")
            return models
        else:
            print(f"\n{ip_port} 没有发现任何 Ollama 模型。")
            return []
    
    except requests.exceptions.RequestException as e:
        print(f"\n无法连接到 {ip_port}，错误: {e}")
        return []

# 让用户输入 IP 和端口，例如：127.0.0.1:5084
ip_port = input("请输入 Ollama 服务器的 IP:端口（如 127.0.0.1:5084）：").strip()

# 调用函数获取模型信息
models = get_ollama_models(ip_port)

In [ ]:
import requests
import json
import tkinter as tk
from tkinter import scrolledtext

# 服务器地址
url = "http://127.0.0.1:5084/api/generate" # 请替换为你的 Ollama 服务器地址

def ask_model():
    """向 Ollama 服务器发送请求并显示结果"""
    prompt = entry.get().strip()
    if not prompt:
        return
    
    data = {
        "model": "mistral:latest",
        "prompt": prompt,
        "stream": False
    }

    try:
        response = requests.post(url, json=data)
        response_json = response.json()
        answer = response_json.get("response", "模型未返回结果")

        # 清空输入框并更新输出框
        output_text.config(state=tk.NORMAL)
        output_text.delete("1.0", tk.END)
        output_text.insert(tk.END, answer)
        output_text.config(state=tk.DISABLED)
    except Exception as e:
        output_text.config(state=tk.NORMAL)
        output_text.delete("1.0", tk.END)
        output_text.insert(tk.END, f"请求失败: {str(e)}")
        output_text.config(state=tk.DISABLED)

# 创建 GUI 界面
root = tk.Tk()
root.title("智慧体交互界面")
root.geometry("600x400")

# 输入框
entry_label = tk.Label(root, text="请输入你的问题:")
entry_label.pack(pady=5)
entry = tk.Entry(root, width=60)
entry.pack(pady=5)

# 发送按钮
ask_button = tk.Button(root, text="提问", command=ask_model)
ask_button.pack(pady=5)

# 输出框（滚动文本）
output_text = scrolledtext.ScrolledText(root, width=70, height=15, wrap=tk.WORD, state=tk.DISABLED)
output_text.pack(pady=10)

# 运行 GUI
root.mainloop()